In [1]:
# import cdsapi
# cds = cdsapi.Client(timeout=600, quiet=False, debug=True)
import numpy as np
import os
# import requests
import zipfile
import tarfile
import subprocess
from datetime import date
import xarray as xr
import pandas as pd
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import theilslopes as sens_slope
from datetime import datetime

from itertools import product
from dask.distributed import wait, progress, Client, LocalCluster
from pathlib import Path
from urllib.request import urlopen
from glob import glob


import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# Define directories
ddir = Path('/data/climate_migration/data/era5/')
disdir = Path('/data/climate_migration/data/disasters/')
griddir = Path('/data/climate_migration/shp/global_sociodem_grids/')
tmpdir = ddir / 'tmp'
zsdir = ddir / 'adm1_zonestats'
shpdir = Path('/data/climate_migration/shp/')
gfdrawdir = shpdir / 'global_flood_database' / 'gfd_v1_4'
gfd_fdd_dir = shpdir / 'global_flood_database' / 'flood_duration_025deg'
gfd_fdd_popwtd_dir = shpdir / 'global_flood_database' / 'flood_duration_popwtd_025deg'
gfd_fdd_agwtd_dir = shpdir / 'global_flood_database' / 'flood_duration_agwtd_025deg'
shptmpdir = shpdir / 'tmp'
hrdir = ddir / 'hourly'
daydir = ddir / 'daily'
wkdir = ddir / 'weekly'
mondir = ddir / 'monthly'
yrdir = ddir / 'yearly'

# Define file paths
ADM1 = shpdir / 'gadm36_1.shp'
LANDSCAN = griddir / 'landscan-global-2020.tif'
WORLDCOVER = griddir / 'worldcover_2020_cropland_resampled_LSres_cultivated_km2.tif'
LANDSCAN_ERA5res = griddir / 'landscan-global-2020_ERA5grid.tif'
WORLDCOVER_ERA5res = griddir / 'worldcover_2020_cropland_resampled_025deg_cultivated_km2.tif'
ERA5GRID = shpdir / 'ERA5_025deg_globalgrid.shp'
EMDAT_GC_ADM = shpdir / 'emdat_nat_complex_disasters_adm1_int_2000_2022.gpkg'
SPI6 = mondir / 'spei' / 'ERA5_monthly_1980_2018_calib_1980_2018_spei_pearson_06.nc'
SPI12 = mondir / 'spei' / 'ERA5_monthly_1980_2018_calib_1980_2018_spei_pearson_12.nc'
SPI24 = mondir / 'spei' / 'ERA5_monthly_1980_2018_calib_1980_2018_spei_pearson_24.nc'

# NOTE: including current year can create some weirdness at the tail end of the time-series
SPI_SCALES = [6,12,24]
allyears = [
    '2016','2017','2018',
    '2019','2020','2021'
]

allmonths = [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ]

alldays = [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ]

alltimes = [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ]

all_yrmon = list(product(allyears,allmonths))[:-6]
n_yrmon = len(all_yrmon)
all_ymd = list(product(allyears,allmonths,alldays))
all_ym = list(product(allyears,allmonths))

AVGCLIMATE_Y0 = 1980
AVGCLIMATE_Y1 = 2018
AVGCLIMATE_YRS = [str(y) for y in np.arange(AVGCLIMATE_Y0,AVGCLIMATE_Y1+1)]
AVGCLIMATE_YMD = list(product(AVGCLIMATE_YRS, allmonths, alldays))

GFDYRS = np.arange(2000,2019)

import gc

In [2]:
# View dashboard by forwarding remote port xxxx to local port (e.g. yyyy), open localhost:yyyy in browser
cluster = LocalCluster(
    dashboard_address = "localhost:45285",
    n_workers = 60, 
    local_directory = "/data/tmp/snair/tmp/dask-worker-space"
)
client = Client(cluster)

2023-06-01 09:52:01,586 - distributed.diskutils - INFO - Found stale lock file and directory '/data/tmp/snair/tmp/dask-worker-space/dask-worker-space/worker-dk1bnn23', purging
2023-06-01 09:52:01,586 - distributed.diskutils - INFO - Found stale lock file and directory '/data/tmp/snair/tmp/dask-worker-space/dask-worker-space/worker-i1l3moxm', purging
2023-06-01 09:52:01,587 - distributed.diskutils - INFO - Found stale lock file and directory '/data/tmp/snair/tmp/dask-worker-space/dask-worker-space/worker-sswnnbu3', purging
2023-06-01 09:52:01,587 - distributed.diskutils - INFO - Found stale lock file and directory '/data/tmp/snair/tmp/dask-worker-space/dask-worker-space/worker-bx9sg85r', purging
2023-06-01 09:52:01,587 - distributed.diskutils - INFO - Found stale lock file and directory '/data/tmp/snair/tmp/dask-worker-space/dask-worker-space/worker-nfvyo5h3', purging
2023-06-01 09:52:01,587 - distributed.diskutils - INFO - Found stale lock file and directory '/data/tmp/snair/tmp/dask-w

In [3]:
def top3(da, var_name, calc_dim ='year', square = False):

    def ufunc_top3(y, square = False):
        x = np.sort(y)[-3:]
        if square:
            x = x**2
        return np.array([np.mean(x)])

    da_top3 = xr.apply_ufunc(
        ufunc_top3,  
        da,
        input_core_dims=[[calc_dim]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[da.dtype], 
        kwargs={'square': square}
    ).compute()
    
    return da_top3

def get_yearly_top3(year, df, date_dim = 'year'):
    """
    Function to get the hottest 3 months in a year
    Returns a netcdf file that contains 3 months for each year. (721 X 1440 X 3)
    """
    
    results = {}
    for v, sq in [('t2m', False),
                  ('t2m', True)]:
        sl = top3(df[v].sel(year = year), v, 'year', square = sq)
        if sq:
            sl = sl.rename('t2m_sq')
            v = 't2m_sq'
        results[v] = sl

    sl_final = xr.combine_by_coords(results.values())
    sl_final = sl_final.assign_coords(year = year)
    return sl_final

#### Read Climate Data

In [21]:
# ### Precipitation
# df_clim_tp = xr.open_dataset(str(mondir / 'tp' / 'ERA5_tp_monthly_1959_2021_v1.nc'))
# df_clim_tp_40 = df_clim_tp.sel(time = slice("1959-01-01", '1998-12-01')).copy()
# df_clim_tp_50 = df_clim_tp.sel(time = slice("1959-01-01", '2008-12-01')).copy()

# years = df_clim_tp['time'].dt.year.values
# df_clim_tp = df_clim_tp.assign_coords(year = ('time',years)).swap_dims({'time':'year'})

# years = df_clim_tp_40['time'].dt.year.values
# df_clim_tp_40 = df_clim_tp_40.assign_coords(year = ('time',years)).swap_dims({'time':'year'})

# years = df_clim_tp_50['time'].dt.year.values
# df_clim_tp_50 = df_clim_tp_50.assign_coords(year = ('time',years)).swap_dims({'time':'year'})


In [4]:
# Precipitation
df_clim_tp = xr.open_dataset(str(mondir / 'tp' / 'ERA5_tp_monthly_1959_2021_v1.nc'))
# df_clim_tp = df_clim_tp.sel(time = slice("1989-01-01", '2018-12-01'))
years = df_clim_tp['time'].dt.year.values
df_clim_tp = df_clim_tp.assign_coords(year = ('time',years)).swap_dims({'time':'year'})


#temperature
df_clim = xr.open_dataset(str(mondir / '2t' / 'ERA5_2t_monthly_1959_2021_v1.nc'))
# df_clim = df_clim.sel(time = slice("1989-01-01", '2018-12-01'))
years = df_clim['time'].dt.year.values
df_clim = df_clim.assign_coords(year = ('time',years)).swap_dims({'time':'year'})

#### Get the hottest three months in each year

In [7]:
#This takes a bit of time to run -- probably want to rewrite using client.map

# results = []
# for y in range(1959, 2019):
#     ## 30 years
#     results.append(get_yearly_top3(y, df = df_clim))
    
# df_final = xr.concat(results, dim = 'year')

In [5]:
futures = client.map(get_yearly_top3, 
                          np.arange(1959, 2019), 
                          df = df_clim, 
                          date_dim = 'year'
                         )

df_lists = [f.result() for f in futures]\

df_final = xr.concat(df_lists, dim = 'year')

gc.collect()

#### Get historical normal

In [52]:
### Precip
df_clim_tp_30= df_clim_tp.sel(year = slice("1959", '1988')).copy()
df_clim_tp_30 = df_clim_tp_30.mean('year')

### All temperatures
df_clim_30 = df_clim.sel(year = slice("1959", "1988")).copy()
df_clim_30 = df_clim_30.mean('year')

## Summer temperatures
df_final_30 = df_final.sel(year = slice("1959", "1988")).copy()
df_final_30 = df_final_30.mean('year')

#### Aggregate at different time scales

In [59]:
dfs = []
for start, end in [
    (1999, 2018), 
#     (2009, 2018), 
#     (1995, 2014)
]:
    ### Temperature ###
    
    ### Summer ###
    df_temp = df_final.sel(year = slice(start, end))
    df_temp = df_temp.mean('year')
    df_temp = df_temp.rename({'t2m':f't2m_{start-1}_{end}_max3', 't2m_sq':f't2m_sq_{start-1}_{end}_max3'})
    
    #Add in Anomalies (reference period: 1959 - 1988)
    df_temp[f't2m_{start-1}_{end}_max3_anomaly'] = df_temp[f't2m_{start-1}_{end}_max3'] - df_final_30['t2m']
    df_temp[f't2m_sq_{start-1}_{end}_max3_anomaly']  = df_temp[f't2m_{start-1}_{end}_max3_anomaly']**2
    
    
    ### All ###
    df_normals = df_clim.sel(year=slice(start, end))
    df_normals = df_normals.mean('year')
    df_normals = df_normals.rename({'t2m':f't2m_Avg_{start-1}_{end}', 't2m_sq':f't2m_sq_Avg_{start-1}_{end}'})
    
    #Add in Anomalies (reference period: 1959 - 1988)
    df_normals[f't2m_Avg_{start-1}_{end}_anomaly'] = df_normals[f't2m_Avg_{start-1}_{end}'] - df_clim_30['t2m']
    df_normals[f't2m_sq_Avg_{start-1}_{end}_anomaly']  = df_normals[f't2m_Avg_{start-1}_{end}_anomaly']**2

#     df_temp = df_temp.merge(df_normals)
    
    ### Precip ####

    ### All ###
    df_precip = df_clim_tp.sel(year = slice(start, end))
    df_precip  =df_precip.mean('year')
    df_precip = df_precip.rename({'tp':f'tp_Avg_{start-1}_{end}', 'tp_sq':f'tp_sq_Avg_{start-1}_{end}'})
    
     #Add in Anomalies (reference period: 1959 - 1988)
    df_precip[f'tp_Avg_{start-1}_{end}_anomaly'] = df_precip[f'tp_Avg_{start-1}_{end}'] - df_clim_tp_30['tp']
    df_precip[f'tp_sq_Avg_{start-1}_{end}_anomaly']  = df_precip[f'tp_Avg_{start-1}_{end}_anomaly']**2
    
#     df_temp  = df_temp.merge(df_precip)
    
    dfs = [df_temp, df_normals, df_precip]
#     df_temp = None
#     df_normals = None
#     df_precip = None
    
    
df_main = xr.merge(dfs)

gc.collect()

42

#### Add in Degree Days

##### To do: add in degree days construction code from Nick's script

In [74]:
dfs = []
for dd_cutoff in [23, 25]:
    print(dd_cutoff)
    df_dd = xr.open_dataset(f"/data/climate_migration/data/era5/monthly/degree_days/ERA5_dd_{dd_cutoff}_monthly_1980_2021.nc")
    for start, end in [(1999, 2018), (2009, 2018)]:
        print(start, end)
        df_dd_temp = df_dd.sel(date= slice(np.datetime64(f"{start}-01-01"), np.datetime64(f"{end}-12-01")))
        df_dd_temp = df_dd_temp.mean('date')
        df_dd_temp = df_dd_temp.rename({x:x+f'_Avg_{start-1}_{end}' for x in df_dd_temp.data_vars})
        
        dfs.append(df_dd_temp)

23
1999 2018
2009 2018
25
1999 2018
2009 2018


#### Merge everything together, and save

In [76]:
df_dd_main = xr.merge(dfs)
df_dd_main = df_dd_main.rename({'latitude':'lat', 'longitude':'lon'})
df_all = xr.merge([df_main, df_dd_main])
df_all = df_all.rename({v:v.replace('t2m_', '') for v in df_all.data_vars if 'dd' in v})

### Updated to v5 on 1st June 2023
df_all.to_netcdf("/data/climate_migration/data/master/climate_master_monthly_v5.nc")
df_all.to_dataframe().to_csv("/data/climate_migration/data/master/climate_master_monthly_v5.csv")

#### Create GADM1 level population weighted aggregates

In [251]:
# df_clim = xr.open_dataset(str(mondir / '2t' / 'ERA5_2t_monthly_1959_2021_v1.nc'))
# df_clim = df_clim.sel(time = slice("1995-01-01", '2014-12-01'))
# years = df_clim['time'].dt.year.values
# df_clim = df_clim.assign_coords(year = ('time',years)).swap_dims({'time':'year'})
# df_clim = df_clim.mean('year')
# df_clim.to_netcdf("/data/climate_migration/data/master/temp_1995_2015.nc")

In [257]:
# command2 = ['~/bin/exactextract', 
#     f'-r "tas:NETCDF:/data/climate_migration/data/master/temp_1995_2015.nc:t2m"', 
#     '-r "pop:/data/climate_migration/shp/global_sociodem_grids/landscan-global-2020.tif"', 
#     '-p /data/climate_migration/shp/gadm36_1.shp -f GID_1', 
#     f'-s "t2m_Avg_1995_2015_pop_mean=weighted_mean(tas,pop)"', 
#     f'-o /data/climate_migration/data/master/gadm1/t2m_Avg_1995_2015_mean_popweighted.csv']

# os.system(' '.join(command2))

Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute


0

In [16]:
for c in ['t2m_1998_2018_max3', 't2m_Avg_1998_2018', 
          't2m_2008_2018_max3', 't2m_Avg_2008_2018', 
          't2m_1994_2014_max3', 't2m_Avg_1994_2014']:

    print(c)
    command2 = ['~/bin/exactextract', 
        f'-r "tas:NETCDF:/data/climate_migration/data/master/climate_master_monthly_v4.nc:{c}"', 
        '-r "pop:/data/climate_migration/shp/global_sociodem_grids/landscan-global-2020.tif"', 
        '-p /data/climate_migration/shp/gadm36_1.shp -f GID_1', 
        f'-s "{c}_pop_mean=weighted_mean(tas,pop)"', 
        f'-o /data/climate_migration/data/master/gadm1/{c}_mean_popweighted.csv']
    
    os.system(' '.join(command2))

t2m_1998_2018_max3


Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute


t2m_Avg_1998_2018


Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute


t2m_2008_2018_max3


...

Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute


t2m_1994_2014_max3


Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute


t2m_Avg_1994_2014


Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute


In [19]:
from functools import reduce
cols = ['t2m_1998_2018_max3', 't2m_Avg_1998_2018', 't2m_2008_2018_max3', 't2m_Avg_2008_2018', 't2m_1994_2014_max3', 
        't2m_Avg_1994_2014']
inpath = "/data/climate_migration/data/master/gadm1"
dfs = [pd.read_csv(f"{inpath}/{c}_mean_popweighted.csv") for c in cols]
dfs = reduce(lambda x, y: pd.merge(x, y, on = 'GID_1'), dfs)
dfs.to_csv(f"{inpath}/gadm1_popweighted.csv", index = False)

In [20]:
dfs

,GID_1,t2m_1998_2018_max3_pop_mean,t2m_Avg_1998_2018_pop_mean,t2m_2008_2018_max3_pop_mean,t2m_Avg_2008_2018_pop_mean,t2m_1994_2014_max3_pop_mean,t2m_Avg_1994_2014_pop_mean
0,AFG.1_1,16.300806,4.806813,16.576187,5.084758,15.949899,4.426816
1,AFG.2_1,22.319508,11.891557,22.781633,12.101844,22.062420,11.591336
2,AFG.3_1,21.904463,10.907340,22.075661,11.049294,21.522558,10.472997
3,AFG.4_1,28.319477,16.817507,28.551853,16.888760,28.110586,16.560560
4,AFG.5_1,13.187573,1.156556,13.248713,1.142758,12.985700,0.914253
...,...,...,...,...,...,...,...
3605,ZWE.6_1,24.413267,21.193567,24.690687,21.352455,24.333332,21.123974
3606,ZWE.7_1,24.690609,21.212187,24.874716,21.467167,24.526571,21.066053
3607,ZWE.8_1,25.415670,21.944431,25.598915,22.079290,25.360188,21.877876
3608,ZWE.9_1,25.013626,21.237770,25.144106,21.422998,24.885050,21.104868
